In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

2023-03-01 14:07:32.490413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Determine the number of unique values in each column.
application_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME", "ORGANIZATION"])

In [7]:
grouping = application_df.groupby("ASK_AMT")["APPLICATION_TYPE"].count()
grouping

ASK_AMT
5000          25398
5001              1
5006              2
5016              1
5037              1
              ...  
3391919220        1
4653011914        1
5591584994        1
8556638692        1
8597806340        1
Name: APPLICATION_TYPE, Length: 8747, dtype: int64

In [10]:
import keras_tuner as kt

ModuleNotFoundError: No module named 'keras_tuner'

In [11]:
pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 2.8 MB/s eta 0:00:0000:01


Note: you may need to restart the kernel to use updated packages.


In [12]:
pip show keras_tuner

Name: keras-tuner
Version: 1.3.0
Summary: A Hyperparameter Tuning Library for Keras
Home-page: https://github.com/keras-team/keras-tuner
Author: The KerasTuner authors
Author-email: 
License: Apache License 2.0
Location: /Users/ama/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages
Requires: ipython, kt-legacy, packaging, requests, tensorflow
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
app_types = application_df.APPLICATION_TYPE.value_counts()
app_types

In [ ]:
# Determine which values to replace if counts are less than ...?

replace_application = list(app_types[app_types <20000].index)
replace_application

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
application_df.dtypes

In [ ]:
application_df[['INCOME_FROM','INCOME_TO']] = application_df['INCOME_AMT'].str.split('-',expand=True)
application_df.dtypes

# Replace 0 on single column
application_df["INCOME_TO"] = application_df["INCOME_TO"].fillna(0) 

application_df["INCOME_TO"] = application_df["INCOME_TO"].replace(to_replace ='[M]', value = '000000', regex = True)
#application_df.head(20)
application_df.dtypes
application_df['INCOME_TO'] = application_df['INCOME_TO'].astype(str).astype(int)
application_df = application_df.drop(columns=["INCOME_FROM", "INCOME_AMT"])

In [ ]:
# Look at CLASSIFICATION value counts for binning
cls_counts = application_df.CLASSIFICATION.value_counts()
cls_counts.head()

In [ ]:
# Determine which values to replace if counts are less than ..?
replace_class = list(cls_counts[cls_counts <700].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

In [ ]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_cat

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
number_input_features = X_train.shape[1]
number_input_features

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer1,
                         input_dim = number_input_features,
                         activation= "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer2, 
                             activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, 
                             activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
pip show tensorflow

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

In [ ]:
from sklearn.linear_model import LogisticRegression
# define logistic regression model
log_classifier = LogisticRegression(solver='lbfgs', max_iter=200)

#Train the model
log_classifier.fit(X_train, y_train)
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test, y_pred):.3f}")

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
#Train the model
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))
print(f" Logistic regression model accuracy: {accuracy_score(y_test, y_pred):.3f}")

In [ ]:

from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = GaussianNB()
clf4 = SVC(probability=True)

eclf = VotingClassifier(estimators=[('LR', clf1), ('RF', clf2), ('GNB', clf3), ('SVC', clf4)],
                        voting='soft', weights=[1,2,1,1])

eclf.fit(X_train, y_train)